# 🧹 Steam Data Cleaning Notebook

This notebook loads the raw scraped data, drops unused columns, and cleans the remaining fields into analysis-ready form.


In [1]:
import pandas as pd
import re

# Load raw data
df = pd.read_csv('raw_data.csv')
print('Original shape:', df.shape)
df.head(3)

Original shape: (8948, 9)


,title,release_date,price,is_free,tags,publisher,developer,review_summary,platforms
0,PUBG: BATTLEGROUNDS,"21 Dec, 2017",Free,True,"Survival, Shooter, Battle Royale, Multiplayer,...","KRAFTON, Inc.",PUBG Corporation,"69% of the 14,380 user reviews in the last 30 ...",NaN
1,Europa Universalis V,"4 Nov, 2025","₹3,259.00",False,"Resource Management, 4X, Grand Strategy, Tradi...",Paradox Interactive,Paradox Tinto,"77% of the 1,695 user reviews in your language...",NaN
2,ARC Raiders,"30 Oct, 2025","₹2,467.00",False,"Extraction Shooter, PvP, PvE, Third-Person Sho...",Embark Studios,Embark Studios,"90% of the 30,474 user reviews in your languag...",NaN


In [2]:
# Drop the 'platforms' column if present
if 'platforms' in df.columns:
    df = df.drop(columns=['platforms'])

print('Columns after dropping:', df.columns.tolist())
df.head(3)

Columns after dropping: ['title', 'release_date', 'price', 'is_free', 'tags', 'publisher', 'developer', 'review_summary']


,title,release_date,price,is_free,tags,publisher,developer,review_summary
0,PUBG: BATTLEGROUNDS,"21 Dec, 2017",Free,True,"Survival, Shooter, Battle Royale, Multiplayer,...","KRAFTON, Inc.",PUBG Corporation,"69% of the 14,380 user reviews in the last 30 ..."
1,Europa Universalis V,"4 Nov, 2025","₹3,259.00",False,"Resource Management, 4X, Grand Strategy, Tradi...",Paradox Interactive,Paradox Tinto,"77% of the 1,695 user reviews in your language..."
2,ARC Raiders,"30 Oct, 2025","₹2,467.00",False,"Extraction Shooter, PvP, PvE, Third-Person Sho...",Embark Studios,Embark Studios,"90% of the 30,474 user reviews in your languag..."


## Clean Columns

- release_date → datetime
- price → numeric
- tags → list
- review_summary → percent + count
- publisher/developer → strip whitespace


In [3]:
# --- Clean release_date ---
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# --- Clean price ---
def parse_price(p):
    if pd.isna(p):
        return None
    p = str(p)
    if 'Free' in p:
        return 0.0
    digits = re.sub(r'[^0-9.]', '', p)
    return float(digits) if digits else None

df['price_clean'] = df['price'].apply(parse_price)

# --- Clean tags ---
df['tags_list'] = df['tags'].dropna().apply(lambda x: [t.strip() for t in str(x).split(',')])

# --- Parse review_summary ---
def parse_review(text):
    if pd.isna(text):
        return pd.Series([None, None])
    match = re.search(r'(\d+)% of the ([\d,]+)', str(text))
    if match:
        percent = int(match.group(1))
        count = int(match.group(2).replace(',', ''))
        return pd.Series([percent, count])
    return pd.Series([None, None])

df[['review_percent', 'review_count']] = df['review_summary'].apply(parse_review)

# --- Clean publisher/developer ---
df['publisher'] = df['publisher'].astype(str).str.strip()
df['developer'] = df['developer'].astype(str).str.strip()

df.head(5)

,title,release_date,price,is_free,tags,publisher,developer,review_summary,price_clean,tags_list,review_percent,review_count
0,PUBG: BATTLEGROUNDS,2017-12-21,Free,True,"Survival, Shooter, Battle Royale, Multiplayer,...","KRAFTON, Inc.",PUBG Corporation,"69% of the 14,380 user reviews in the last 30 ...",0.0,"[Survival, Shooter, Battle Royale, Multiplayer...",69.0,14380.0
1,Europa Universalis V,2025-11-04,"₹3,259.00",False,"Resource Management, 4X, Grand Strategy, Tradi...",Paradox Interactive,Paradox Tinto,"77% of the 1,695 user reviews in your language...",3259.0,"[Resource Management, 4X, Grand Strategy, Trad...",77.0,1695.0
2,ARC Raiders,2025-10-30,"₹2,467.00",False,"Extraction Shooter, PvP, PvE, Third-Person Sho...",Embark Studios,Embark Studios,"90% of the 30,474 user reviews in your languag...",2467.0,"[Extraction Shooter, PvP, PvE, Third-Person Sh...",90.0,30474.0
3,Counter-Strike 2,2012-08-21,Free,True,"FPS, Shooter, Multiplayer, Competitive, Action...",Valve,Valve,"81% of the 83,999 user reviews in the last 30 ...",0.0,"[FPS, Shooter, Multiplayer, Competitive, Actio...",81.0,83999.0
4,Apex Legends™,2020-11-04,Free,True,"Free to Play, Multiplayer, Battle Royale, FPS,...",Electronic Arts,Respawn,"66% of the 4,670 user reviews in the last 30 d...",0.0,"[Free to Play, Multiplayer, Battle Royale, FPS...",66.0,4670.0


In [ ]:
# # Save cleaned data
# df.to_csv('clean_data.csv', index=False)
# print('Cleaned dataset saved as clean_data.csv')
# print('Final shape:', df.shape)

Cleaned dataset saved as clean_data.csv
Final shape: (8948, 12)


In [ ]:
# --- Drop redundant raw columns ---
cols_to_drop = []
if 'price' in df.columns:
    cols_to_drop.append('price')
if 'review_summary' in df.columns:
    cols_to_drop.append('review_summary')

df = df.drop(columns=cols_to_drop, errors='ignore')

# --- Create review_category from review_percent ---
def categorize_review(pct):
    if pd.isna(pct):
        return None
    if pct <= 40:
        return 'Negative'
    elif pct <= 70:
        return 'Mixed'
    else:
        return 'Positive'

df['review_category'] = df['review_percent'].apply(categorize_review)

# --- Create price_bucket from price_clean ---
def bucket_price(val):
    if pd.isna(val):
        return None
    if val == 0:
        return 'Free'
    elif val <= 500:
        return 'Low'
    elif val <= 1500:
        return 'Medium'
    else:
        return 'High'

df['price_bucket'] = df['price_clean'].apply(bucket_price)

# --- Count number of tags per game ---
df['tag_count'] = df['tags_list'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# # --- Save enriched dataset ---
# df.to_csv("clean_data_enriched.csv", index=False)

# print("Enriched dataset saved as clean_data_enriched.csv")
df.head(5)


Enriched dataset saved as clean_data_enriched.csv


,title,release_date,is_free,tags,publisher,developer,price_clean,tags_list,review_percent,review_count,review_category,price_bucket,tag_count
0,PUBG: BATTLEGROUNDS,2017-12-21,True,"Survival, Shooter, Battle Royale, Multiplayer,...","KRAFTON, Inc.",PUBG Corporation,0.0,"[Survival, Shooter, Battle Royale, Multiplayer...",69.0,14380.0,Mixed,Free,20
1,Europa Universalis V,2025-11-04,False,"Resource Management, 4X, Grand Strategy, Tradi...",Paradox Interactive,Paradox Tinto,3259.0,"[Resource Management, 4X, Grand Strategy, Trad...",77.0,1695.0,Positive,High,20
2,ARC Raiders,2025-10-30,False,"Extraction Shooter, PvP, PvE, Third-Person Sho...",Embark Studios,Embark Studios,2467.0,"[Extraction Shooter, PvP, PvE, Third-Person Sh...",90.0,30474.0,Positive,High,20
3,Counter-Strike 2,2012-08-21,True,"FPS, Shooter, Multiplayer, Competitive, Action...",Valve,Valve,0.0,"[FPS, Shooter, Multiplayer, Competitive, Actio...",81.0,83999.0,Positive,Free,20
4,Apex Legends™,2020-11-04,True,"Free to Play, Multiplayer, Battle Royale, FPS,...",Electronic Arts,Respawn,0.0,"[Free to Play, Multiplayer, Battle Royale, FPS...",66.0,4670.0,Mixed,Free,20


In [6]:
# Drop raw tags and developer
df = df.drop(columns=['tags', 'developer'], errors='ignore')

# Extract release year (and month if needed)
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Drop full release_date if you only want year/month
df = df.drop(columns=['release_date'], errors='ignore')

# Drop rows with too many missing values (keep only rows with >=50% filled)
df = df.dropna(thresh=int(df.shape[1] * 0.5))

print("Final columns:", df.columns.tolist())
print("Final shape:", df.shape)


Final columns: ['title', 'is_free', 'publisher', 'price_clean', 'tags_list', 'review_percent', 'review_count', 'review_category', 'price_bucket', 'tag_count', 'release_year', 'release_month']
Final shape: (8947, 12)


In [7]:
# --- Drop tag_count if present ---
df = df.drop(columns=['tag_count'], errors='ignore')

# --- Fill all missing values with 'NA' ---
df = df.fillna("NA")

# --- Save final dataset ---
df.to_csv("clean_data_final.csv", index=False)

print("Final dataset saved as clean_data_final.csv")
print("Shape:", df.shape)
df.head(5)


Final dataset saved as clean_data_final.csv
Shape: (8947, 11)


,title,is_free,publisher,price_clean,tags_list,review_percent,review_count,review_category,price_bucket,release_year,release_month
0,PUBG: BATTLEGROUNDS,True,"KRAFTON, Inc.",0.0,"[Survival, Shooter, Battle Royale, Multiplayer...",69.0,14380.0,Mixed,Free,2017.0,12.0
1,Europa Universalis V,False,Paradox Interactive,3259.0,"[Resource Management, 4X, Grand Strategy, Trad...",77.0,1695.0,Positive,High,2025.0,11.0
2,ARC Raiders,False,Embark Studios,2467.0,"[Extraction Shooter, PvP, PvE, Third-Person Sh...",90.0,30474.0,Positive,High,2025.0,10.0
3,Counter-Strike 2,True,Valve,0.0,"[FPS, Shooter, Multiplayer, Competitive, Actio...",81.0,83999.0,Positive,Free,2012.0,8.0
4,Apex Legends™,True,Electronic Arts,0.0,"[Free to Play, Multiplayer, Battle Royale, FPS...",66.0,4670.0,Mixed,Free,2020.0,11.0


In [8]:
import pandas as pd
import numpy as np
import calendar

# Load the latest cleaned data that still has tags_list
df = pd.read_csv("clean_data_final.csv")

# 1) Rename price_clean -> price (cleaner, standard)
if "price_clean" in df.columns:
    df = df.rename(columns={"price_clean": "price"})

# 2) Ensure release_year is an integer (no .0)
if "release_year" in df.columns:
    df["release_year"] = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")

# 3) Convert release_month to month abbreviation (1->Jan, 2->Feb, ...), keep NA if missing
def to_month_abbr(m):
    if pd.isna(m):
        return pd.NA
    try:
        mi = int(m)
        return calendar.month_abbr[mi] if 1 <= mi <= 12 else pd.NA
    except Exception:
        return pd.NA

if "release_month" in df.columns:
    df["release_month"] = df["release_month"].apply(to_month_abbr)

# 4) Price floor/ceil to nearest 500 (numeric, not strings, for further ops)
def price_floor_500(val):
    if pd.isna(val):
        return pd.NA
    try:
        v = int(round(float(val)))
        return (v // 500) * 500
    except Exception:
        return pd.NA

def price_ceil_500(val):
    if pd.isna(val):
        return pd.NA
    try:
        v = int(round(float(val)))
        return ((v + 499) // 500) * 500
    except Exception:
        return pd.NA

df["price_floor"] = df["price"].apply(price_floor_500)
df["price_ceil"]  = df["price"].apply(price_ceil_500)

# 5) Review outliers: mark by count threshold and store outlier titles
OUTLIER_THRESHOLD = 30000
df["review_outlier"] = df["review_count"].apply(
    lambda x: "Yes" if (pd.notna(x) and float(x) > OUTLIER_THRESHOLD) else "No"
)

# A helper column to keep the title only for outliers (empty otherwise)
df["outlier_title"] = np.where(df["review_outlier"] == "Yes", df["title"], "")

# Optional: keep a clean numeric version of price as int for plotting
df["price_int"] = df["price"].apply(lambda x: int(round(float(x))) if pd.notna(x) else pd.NA).astype("Int64")

# Save the grouped dataset (v3)
df.to_csv("clean_data_grouped_v3.csv", index=False)
print("Saved clean_data_grouped_v3.csv")
print("Outlier titles:", df.loc[df["review_outlier"]=="Yes", "title"].tolist())


Saved clean_data_grouped_v3.csv
Outlier titles: ['ARC Raiders', 'Counter-Strike 2', 'Battlefield™ 6', 'Hollow Knight: Silksong']


In [9]:
import pandas as pd
import numpy as np
import ast

# Load the same base file to ensure tags_list is present
df_tags = pd.read_csv("clean_data_final.csv")

# Ensure we carry over outlier logic by recalculating (or merge from grouped if preferred)
OUTLIER_THRESHOLD = 30000
if "review_outlier" not in df_tags.columns:
    df_tags["review_outlier"] = df_tags["review_count"].apply(
        lambda x: "Yes" if (pd.notna(x) and float(x) > OUTLIER_THRESHOLD) else "No"
    )
df_tags["outlier_title"] = np.where(df_tags["review_outlier"] == "Yes", df_tags["title"], "")

# Robust parsing of tags_list handling NA, stringified lists, and comma strings
def parse_tags(x):
    if pd.isna(x):
        return []
    if isinstance(x, str):
        s = x.strip()
        if s == "NA" or s == "":
            return []
        # Case: stringified Python list e.g. "['RPG','Action']"
        if s.startswith("[") and s.endswith("]"):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, list):
                    return [str(t).strip() for t in parsed if str(t).strip()]
                return []
            except Exception:
                return []
        # Case: comma-separated string e.g. "RPG, Action, Adventure"
        return [t.strip() for t in s.split(",") if t.strip()]
    # Any other type: fallback
    try:
        return list(x) if isinstance(x, (list, tuple, set)) else []
    except Exception:
        return []

# Parse tags
if "tags_list" in df_tags.columns:
    df_tags["tags_list"] = df_tags["tags_list"].apply(parse_tags)
else:
    # If missing, create empty lists
    df_tags["tags_list"] = [[] for _ in range(len(df_tags))]

# Explode into multiple rows (one per tag)
df_exploded = df_tags.explode("tags_list").rename(columns={"tags_list": "tag"})

# Drop empty tags
df_exploded = df_exploded[df_exploded["tag"].notna() & (df_exploded["tag"].astype(str).str.strip() != "")]

# Standardize tag text (optional)
df_exploded["tag"] = df_exploded["tag"].astype(str).str.strip()

# Optional: bring clean naming and numeric columns consistent with grouped dataset
if "price_clean" in df_exploded.columns and "price" not in df_exploded.columns:
    df_exploded = df_exploded.rename(columns={"price_clean": "price"})

# Ensure int price column for plotting
df_exploded["price_int"] = df_exploded["price"].apply(
    lambda x: int(round(float(x))) if pd.notna(x) else pd.NA
).astype("Int64")

# Save exploded dataset (v3)
df_exploded.to_csv("clean_data_exploded_v3.csv", index=False)
print("Saved clean_data_exploded_v3.csv")
print("Exploded rows:", len(df_exploded))
print("Unique tags:", df_exploded["tag"].nunique())
print("Outlier titles in exploded:", df_exploded.loc[df_exploded["review_outlier"]=="Yes","title"].unique().tolist())


Saved clean_data_exploded_v3.csv
Exploded rows: 130223
Unique tags: 445
Outlier titles in exploded: ['ARC Raiders', 'Counter-Strike 2', 'Battlefield™ 6', 'Hollow Knight: Silksong']
